In [1]:
import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
# import keras_metrics as km
from keras import regularizers, optimizers
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import accuracy_score
import math

In [2]:
data_dir = "/content/drive/MyDrive/video_data"
img_height , img_width = 64, 64
seq_len = 16

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
classes = ["fall", "fall_down", "grab", "swallow", "walk","sit"]

In [5]:
def frames_extraction(video_path):
    frames_list = []
     
    vidcap = cv2.VideoCapture(video_path)
    total_frames = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)

    frames_step = total_frames//seq_len
 
    for j in range(seq_len):
        #here, we set the parameter 1 which is the frame number to the frame (i*frames_step)
        vidcap.set(1,j*frames_step)
        success,image = vidcap.read()  
        #save your image
        if success:
          image = cv2.resize(image, (img_height, img_width))
          frames_list.append(image)
        else:
          print("Defected frame {}".format(video_path))
          break

    return frames_list

In [6]:
def create_data(input_dir):
    X = []
    Y = []
     
    classes_list = os.listdir(input_dir)
     
    for c in classes_list:
        print(c)
        files_list = os.listdir(os.path.join(input_dir, c))
        for f in files_list:
            frames = frames_extraction(os.path.join(os.path.join(input_dir, c), f))
            if len(frames) == seq_len:
                X.append(frames)
             
                y = [0]*len(classes)
                y[classes.index(c)] = 1
                Y.append(y)
     
    X = np.asarray(X)
    Y = np.asarray(Y)
    return X, Y

In [ ]:
X, Y = create_data(data_dir)
 
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, shuffle=True, random_state=0)

walk
fall_down
swallow
Defected frame /content/drive/MyDrive/video_data/swallow/70.mov
Defected frame /content/drive/MyDrive/video_data/swallow/79_crop (online-video-cutter.com).mp4
Defected frame /content/drive/MyDrive/video_data/swallow/80 (online-video-cutter.com).mp4
Defected frame /content/drive/MyDrive/video_data/swallow/81_crop (online-video-cutter.com).mp4
Defected frame /content/drive/MyDrive/video_data/swallow/82_crop (online-video-cutter.com).mp4
Defected frame /content/drive/MyDrive/video_data/swallow/83_crop (online-video-cutter.com).mp4


In [ ]:
model = Sequential()
model.add(ConvLSTM2D(filters = 32, kernel_size = (3, 3), return_sequences = True, padding = 'same', data_format = "channels_last", input_shape = (seq_len, img_height, img_width, 3)))
model.add(ConvLSTM2D(filters = 32, kernel_size = (3, 3), strides = (2,2), return_sequences = True, padding = 'same', data_format = "channels_last"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(ConvLSTM2D(filters = 64, kernel_size = (3, 3), return_sequences = True, padding = 'same', data_format = "channels_last"))
model.add(ConvLSTM2D(filters = 64, kernel_size = (3, 3), strides = (2,2), return_sequences = False, padding = 'same', data_format = "channels_last"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(6, activation = "softmax"))
 
model.summary()

In [ ]:
def schedular(epoch):
  if epoch < 10:
    return 0.001
  else:
    return 0.001 * math.exp(0.1*(10-epoch))

In [ ]:
opt = optimizers.Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])
 
earlystop = EarlyStopping(patience=7)
callbacks = tf.keras.callbacks.LearningRateScheduler(schedular)
 
history = model.fit(x = X_train, y = y_train, epochs=20, batch_size = 32, shuffle=True, validation_split=0.2, callbacks=[callbacks])

import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'val','accuracy','val_accuracy'])
plt.show()


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/20
19/19 [==============================] - 1171s 60s/step - loss: 3.6596 - accuracy: 0.2654 - val_loss: 2.7561 - val_accuracy: 0.2914
Epoch 2/20
19/19 [==============================] - 1139s 60s/step - loss: 1.3953 - accuracy: 0.4641 - val_loss: 1.7565 - val_accuracy: 0.3377
Epoch 3/20
19/19 [==============================] - 1148s 61s/step - loss: 1.2063 - accuracy: 0.5596 - val_loss: 1.4705 - val_accuracy: 0.3709
Epoch 4/20
19/19 [==============================] - 1150s 61s/step - loss: 1.0857 - accuracy: 0.6414 - val_loss: 1.4962 - val_accuracy: 0.4437
Epoch 5/20
19/19 [==============================] - 1159s 61s/step - loss: 0.7918 - accuracy: 0.7009 - val_loss: 1.7898 - val_accuracy: 0.3642
Epoch 6/20
19/19 [==============================] - 1159s 61s/step - loss: 0.7042 - accuracy: 0.7481 - val_loss: 1.3809 - val_accuracy: 0.5166
Epoch 7/20
19/19 [==============================] - 1160s 61s/step - loss: 0.6038 - accuracy: 0.7661 - val_loss: 1.8266 - val_accuracy: 0.3841

In [13]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test) 
y_pred = np.argmax(y_pred, axis = 1)
# y_test = np.argmax(y_test, axis = 1)
 
print(classification_report(y_test, y_pred))

print('accuracy score : {}'.format(accuracy_score(y_pred, y_test)))


              precision    recall  f1-score   support

           0       0.60      0.40      0.48        15
           1       0.71      0.53      0.60        55
           2       0.47      0.58      0.52        33
           3       0.45      0.55      0.49        33
           4       0.42      0.44      0.43        34
           5       0.45      0.53      0.49        19

    accuracy                           0.51       189
   macro avg       0.52      0.50      0.50       189
weighted avg       0.54      0.51      0.52       189

accuracy score : 0.5132275132275133
